# Create a simulator that reports the machine's sensor values

We don't have a real machine to monitor, so let's simulate one.
For this we need a few components:

* a machine that keeps running (M1)
* a sensor attached to that machine (S1)
* an API that we can query from our application (M-API)

# Tasks

1. Run the code below
2. Query the machine using your web browser
3. Try to run different configurations and see if this changes your output

# The machine and it's sensor

In [2]:
import numpy as np
from threading import Thread
from datetime import datetime as dt
from time import sleep


# Configuration 1: working ok, intermittent failure
shape, scale = 1., .1  
# Configuration 2: fail more often
# shape, scale = 5., .1  
# Configuration 3: constantly failing
# shape, scale = 10., .1   

# sensor buffer
size = 1000
STATE = [(None, 0)] * size

def sensor():
    global STATE
    rnd = np.random.default_rng()
    while True:
        data = rnd.gamma(shape, scale, size)
        for i, value in enumerate(data):
            record = (dt.now().isoformat(), value)
            STATE = STATE[1:] + [record]
            sleep(.1)

def machine():
    # we use a Thread to run the machine "in the background"
    t = Thread(target=sensor)
    t.start()

# The sensor API

The sensor API is very simple

* `http://localhost:5000/query/10` reports the latest 10 sensor values
* `http://localhost:5000/query/100` reports the latest 100 sensor values
* it can report at most `size=1000` values (we could change this)

In [ ]:
from flask import Flask

def create_app():
    app = Flask("factory")
    
    @app.route("/query/<int:records>")
    def data(records):
        values = STATE[-records:]
        return { 'values': values }
    
    return app


machine()

app = create_app()
app.run()
    
    

 * Serving Flask app 'factory' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Oct/2021 01:49:42] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:43] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:44] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:45] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:46] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:47] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:48] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:53] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:53] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:53] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:54] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:54] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:55] "GET /query/100 HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2021 01:49:55] "GET /